In [0]:
# Databricks Community edition doesn't support secrets, so credentials are stored in a JSON file
# Confirmed with Databricks tech support Marcus de Varona on 6/4/2021
secret = spark.read.json("/FileStore/tables/phData_challenge/secret.json", multiLine=True).first().asDict()

# Snowflake connection options
options = {
  "sfUrl": secret['url'],
  "sfUser": secret['username'],
  "sfPassword": secret['password'],
  "sfDatabase": secret['database'],
  "sfSchema": secret['schema'],
  "sfWarehouse": secret['warehouse']
}

# Query report views
def execute_query(query): 
  df = spark.read \
    .format("snowflake") \
    .options(**options) \
    .option("query", query) \
    .load()
  return df

# Flights by Month

In [0]:
df = execute_query('''
  select
      airline_name as airlines,
      month as month,
      sum(flights_from_airport) as num_flights
  from fligths_by_airline_airport_month
  group by airline_name, month
''')
display(df)

AIRLINES,MONTH,NUM_FLIGHTS
Atlantic Southeast Airlines,2015-02,45138
Atlantic Southeast Airlines,2015-03,54190
US Airways Inc.,2015-06,34300
Southwest Airlines Co.,2015-02,90172
Southwest Airlines Co.,2015-03,109245
American Airlines Inc.,2015-01,44059
American Airlines Inc.,2015-03,45966
American Airlines Inc.,2015-04,44770
JetBlue Airways,2015-03,22590
JetBlue Airways,2015-05,22565


# On Time vs. Late Flights

In [0]:
df = execute_query('''
  select
    airline as airlines,
    on_time_flights as on_time,
    total_flights - on_time_flights as late
  from on_time_pct_by_airline_2015
  order by total_flights desc
''')
display(df)

AIRLINES,ON_TIME,LATE
WN,615248,214534
DL,470010,110431
AA,314007,101449
OO,303626,85038
EV,291460,90720
UA,232407,103287
MQ,141101,55808
US,152297,41926
B6,126687,47871
AS,97096,17516


In [0]:
df = execute_query('''
  select
    airline as airlines,
    on_time_pct as on_time,
    100 - on_time_pct as late
  from on_time_pct_by_airline_2015
  order by on_time_pct
''')
display(df)

AIRLINES,ON_TIME,LATE
NK,63.49,36.51
F9,67.34,32.66
UA,69.23,30.77
MQ,71.66,28.34
B6,72.58,27.42
WN,74.15,25.85
AA,75.58,24.42
EV,76.26,23.74
VX,76.78,23.22
OO,78.12,21.88


# Departure & Arrival Delays

In [0]:
df = execute_query('''
  select
      airline_name as airlines,
      departure_delays as departure,
      arrival_delays as arrival,
      total_delays as total
  from delays_by_airline
  order by total_delays desc
''')
display(df)

AIRLINES,DEPARTURE,ARRIVAL,TOTAL
Southwest Airlines Co.,395439,332201,727640
Delta Air Lines Inc.,200207,178618,378825
United Air Lines Inc.,185714,137201,322915
American Airlines Inc.,155094,156324,311418
Atlantic Southeast Airlines,125056,154305,279361
Skywest Airlines Inc.,120229,152963,273192
American Eagle Airlines Inc.,74397,82823,157220
US Airways Inc.,62565,76285,138850
JetBlue Airways,67361,67778,135139
Spirit Air Lines,36357,38699,75056


# Cancellation Reasons

In [0]:
df = execute_query('''
  select top 10
    airport_code as airports,
    weather,
    airline_carrier,
    national_air_system,
    security
  from cancellation_reasons_by_airport_pivoted
  order by total desc
''')
display(df)

AIRPORTS,WEATHER,AIRLINE_CARRIER,NATIONAL_AIR_SYSTEM,SECURITY
ORD,3463,1269,1853,0
DFW,4023,1248,110,0
LGA,2080,990,1063,1
EWR,1032,323,1346,0
BOS,1827,362,294,0
ATL,1472,574,140,0
DCA,1014,445,359,0
JFK,1089,247,459,0
IAH,944,422,404,0
LAX,489,1066,213,0


# Delay Reasons

In [0]:
df = execute_query('''
  select top 10
    airport_code as airports,
    air_system_delays,
    security_delays,
    airline_delays,
    late_aircraft_delays,
    weather_delays
  from delay_reasons_by_airport
  order by 
    air_system_delays +
    security_delays +
    airline_delays +
    late_aircraft_delays +
    weather_delays
    desc
''')
display(df)

AIRPORTS,AIR_SYSTEM_DELAYS,SECURITY_DELAYS,AIRLINE_DELAYS,LATE_AIRCRAFT_DELAYS,WEATHER_DELAYS
ORD,27604,100,28251,25687,7385
ATL,20336,42,25639,20084,5068
DFW,19890,203,24247,19820,4839
DEN,16921,24,19541,18609,1969
LAX,16465,85,17180,18009,749
IAH,13113,49,14162,11069,1306
LAS,10219,26,12768,12806,568
PHX,10779,247,12877,11400,500
SFO,9971,39,11846,12482,596
LGA,11673,26,8638,10647,1264


# Most Unique Routes

In [0]:
df = execute_query('''
  select *
  from most_unique_routes_airlines
  order by unique_routes desc;
''')
display(df)

AIRLINE_CODE,AIRLINE_NAME,UNIQUE_ROUTES
EV,Atlantic Southeast Airlines,1351
WN,Southwest Airlines Co.,1308
OO,Skywest Airlines Inc.,1245
DL,Delta Air Lines Inc.,919
AA,American Airlines Inc.,656
UA,United Air Lines Inc.,641
MQ,American Eagle Airlines Inc.,403
US,US Airways Inc.,340
B6,JetBlue Airways,324
F9,Frontier Airlines Inc.,313
